In [1]:
#Run any one cell to select a certain url

In [2]:
import getpass
import os

password = getpass.getpass()
command = "sudo -S apt-get install chromium-chromedriver"
os.system('echo %s | %s' % (password, command))
command = "sudo -S apt-get update"
os.system('echo %s | %s' % (password, command))
command = "sudo -S apt-get install chromium-chromedriver"
os.system('echo %s | %s' % (password, command))

# If facing issues with the default firefox driver for webscraping using selenium , run the below code
# !sudo apt-get install chromium-chromedriver
# !sudo apt-get update
# !sudo apt-get install chromium-chromedriver
!pip install selenium
# Here the chrome driver is being used , hence replace firefox with Chrome driver as given below
!pip install webdriver-manager

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(ChromeDriverManager().install(), options=options)

········
You should consider upgrading via the '/home/gokulvamsit/sem4-summer/IndicWiki/Gokul_IndicWiki/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/gokulvamsit/sem4-summer/IndicWiki/Gokul_IndicWiki/bin/python -m pip install --upgrade pip' command.




====== WebDriver manager ======
Current google-chrome version is 89.0.4389
Get LATEST driver version for 89.0.4389
Driver [/home/gokulvamsit/.wdm/drivers/chromedriver/linux64/89.0.4389.23/chromedriver] found in cache


In [3]:
#Alphabetical parse url
#url = "https://www.espncricinfo.com/ci/content/player/country.html?country=6;alpha=A"

In [4]:
from bs4 import BeautifulSoup as bf
import requests
import pandas as pd
import time
from selenium import webdriver

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# driver = webdriver.Firefox()
# driver = webdriver.Chrome(options=options)
driver = wd

In [5]:
def evaluate(html):
  select_divs = html.find_all('div',class_="card overflow-hidden mb-3")
  exist = 0
  location = 0
  for i in range(len(select_divs)):
    check = ""
    try:
      check = select_divs[i].find('p',class_="benton-bold pl-3 pt-4 pb-3 m-0 player-card-header").text
    except:
      k = 1
    if(check == "Career Averages"):
      exist = 1
      location = i
      break
  return exist,select_divs[location]

In [6]:
# Handling images
def get_player_image(player_url):
    # For fixing installation errors on linux: `sudo apt install firefox-geckodriver`
    global driver
    driver.get(player_url)
    time.sleep(2)
    images = driver.find_elements_by_tag_name('img')
    images = [image for image in images if "player-card__face" in image.get_attribute("class")]
    if len(images) < 1:
        return ["image"], [""]
    print(images[0].get_attribute("src"))
    return ["image"], [images[0].get_attribute("src")]

In [7]:
# Get trophy info
def get_major_trophies_information(major_trophies_soup):
    major_trophies_results = {}
    all_h5_tags = major_trophies_soup.find_all('h5', class_="border-bottom-gray-300 m-0 pl-3 pb-2 table-header")
    if all_h5_tags is None:
        return None
    required_h5_tag = all_h5_tags[0]
    for h5_tag in all_h5_tags:
        if str(h5_tag.text).strip() == "In Major Trophies":
            required_h5_tag = h5_tag
            break
    if str(required_h5_tag.text).strip() != "In Major Trophies":
        return ["Major trophies"], [[major_trophies_results]]
    major_trophies_table_head = required_h5_tag.findNext('thead')
    major_trophies_table_headings = major_trophies_table_head.find_all('th')
    major_trophies_table_headings_count = 0
    major_trophies_headings_list = []
    for table_heading in major_trophies_table_headings:
        major_trophies_headings_list.append(table_heading.text)
    major_trophies_table_headings_count = len(major_trophies_headings_list)
    # print('Major Trophies table Head count', major_trophies_table_headings_count)
    current_title = "HOME"
    major_trophies_table_body = major_trophies_table_head.findNext('tbody')
    major_trophies_table_data = major_trophies_table_body.find_all('td')
    t = 0
    for table_data in major_trophies_table_data:
        attribute_number = t % major_trophies_table_headings_count
        if(attribute_number != 0):
            major_trophies_results[current_title][major_trophies_headings_list[attribute_number]] = table_data.text
        else:
            current_title = table_data.text
            major_trophies_results[current_title] = {}
        t += 1
    print(major_trophies_results)
    return ["Major trophies"], [[major_trophies_results]]

In [8]:
# Statistical analysis - Home, Away stats and function call to trophy info
def get_statistical_analysis_and_trophy_info(req):
    analysis_labels = []
    analysis_results = []
    analysis_url = req.url + "/bowling-batting-stats"
    #analysis_url = "https://www.espncricinfo.com/player/virat-kohli-253802/bowling-batting-stats"
    
    # selecting dropdown option
    global driver
    driver.get(analysis_url)
    # loading page
    time.sleep(2)
    dropdowns = driver.find_elements_by_class_name('dropdown-container')
    for dropdown in dropdowns:
        is_format_dropdown = str(dropdown.find_elements_by_tag_name('button')[0].text).strip() in ["Test", "ODI", "T20I", "Test+ODI+T20I"]
        is_role_dropdown = str(dropdown.find_elements_by_tag_name('button')[0].text).strip() in ["Batting", "Bowling", "Fielding", "Allround"]
        if (not is_format_dropdown) and (not is_role_dropdown):
            continue
        required_span_text = "Test+ODI+T20I"
        if is_role_dropdown:
            required_span_text = "Allround"
        dropdown.find_elements_by_tag_name('button')[0].click()
        all_list_items = dropdown.find_elements_by_tag_name('li')
        for list_item in all_list_items:
            span_text = list_item.find_elements_by_tag_name('span')[0].text
            if span_text == required_span_text:
                list_item.click()
                break
    # waiting for changes to load
    time.sleep(2)
    analysis_page = driver.page_source
    analysis_soup = bf(analysis_page, "html.parser")
    
    all_h5_tags = analysis_soup.find_all('h5', class_="border-bottom-gray-300 m-0 pl-3 pb-2 table-header")
    if all_h5_tags is None:
        return None
    required_h5_tag = all_h5_tags[0]
    for h5_tag in all_h5_tags:
        if str(h5_tag.text).strip() == "Home vs Away":
            required_h5_tag = h5_tag
            break
    if str(required_h5_tag.text).strip() != "Home vs Away":
        return get_major_trophies_information(analysis_soup)
    analysis_table_head = required_h5_tag.findNext('thead')
    analysis_table_headings = analysis_table_head.find_all('th')
    analysis_table_headings_count = 0
    analysis_table_headings_list = []
    for table_heading in analysis_table_headings:
        analysis_table_headings_list.append(table_heading.text)
    analysis_table_headings_count = len(analysis_table_headings_list)
    # print('Analysis table Head count', analysis_table_headings_count)
    current_title = "HOME"
    analysis_table_body = analysis_table_head.findNext('tbody')
    analysis_table_data = analysis_table_body.find_all('td')
    t = 0
    for table_data in analysis_table_data:
        attribute_number = t % analysis_table_headings_count
        if(attribute_number != 0):
            analysis_results.append(table_data.text)
            analysis_labels.append(current_title + "_" + analysis_table_headings_list[attribute_number])
        else:
            current_title = table_data.text.upper()
        t += 1
    # for i in range(len(analysis_labels)):
    #     print(f'{analysis_labels[i]} : {analysis_results[i]}')
    trophy_label, trophy_result = get_major_trophies_information(analysis_soup)
    return analysis_labels + trophy_label, analysis_results + trophy_result

In [9]:
def find_relations(Relations):
    relations_container = Relations.find_next('div')
    individual_relations = relations_container.find_all('span')
    total_relations = list()
    for i in range(len(individual_relations)):
        name = individual_relations[i].find('h5').text
        bond = individual_relations[i].find('p').text
        total_relations.append([name,bond])
    return total_relations

In [1]:
#Get relevant information related to a player using each player's url
def load(req,final_list_players,ind,u,debut_url):
    lab = list()
    result = list()
    soup = bf(req.text,"html.parser")
    print(soup.title)
    search_tag_container = soup.find_all('h5',class_='player-card-description gray-900')
    count = 0
    for job_elem in search_tag_container:
        result.append(job_elem.text)
        count+=1
    search_tag_container = soup.find_all('p',class_="text-uppercase gray-700 mb-0 pb-0-5 player-card-heading")
    for job_elem in search_tag_container:
        lab.append(job_elem.text)
    player_role = "xyz"
    for i in range(len(lab)):
        if "role" in lab[i].lower():
            player_role = result[i].lower()
            break
    if player_role == "xyz":
        return
    ##################################
    valid, bat_bowl_soup = evaluate(soup)
    if valid != 1:
        return
    search_tag_container = bat_bowl_soup.findAll('thead')
    match_averages = []
    runs_scored_as_batsman = 0
    balls_bowled_as_bowler = 0
    for m in range(len(search_tag_container)):
        actual_heading = search_tag_container[m].findPrevious('h5').text
        current_string = "_"
        if "Bowling" in actual_heading:
            current_string = "Bowling_"
        elif "Batting" in actual_heading:
            current_string = "Batting_"
        else:
            continue
        match_records = {}
        tag_content = search_tag_container[m].find_all('th')
        head = 0
        table_labels = list()
        table_labels_1 =list()
        for job_elem in tag_content:
            table_labels.append(job_elem.text)
            head += 1
        print('Head count',head)

        tag_content = search_tag_container[m].findNext('tbody').find_all('td')
        track_head_label = 0
        current_format = ""
        for job_elem in tag_content:
            attribute_number = track_head_label % head
            if(attribute_number != 0):
                result.append(job_elem.text)
                label_text = table_labels[attribute_number]
                lab.append(current_string + current_format + '_' + label_text)
                count += 1
                if label_text.lower() == "mat":
                    match_records[current_format.lower()] = job_elem.text
                if current_string == "Batting_" and label_text.lower() == "runs":
                    try:
                        runs_scored_as_batsman += (int(job_elem.text))
                    except:
                        pass
                elif current_string == "Bowling_" and label_text.lower() == "balls":
                    try:
                        balls_bowled_as_bowler += (int(job_elem.text))
                    except:
                        pass
            else:
                current_format = job_elem.text
            track_head_label += 1
        match_averages.append(match_records)
        
    if(len(match_averages) != 2):
        return
    internationals = 0
    non_internationals = 0
    for key in match_averages[0]:
        if key in match_averages[1]:
            if key == "test" or key == "odi" or key == "t20i":
                try:
                    internationals += max(int(match_averages[0][key]), int(match_averages[1][key]))
                except:
                    pass
            else:
                try:
                    non_internationals += max(int(match_averages[0][key]), int(match_averages[1][key]))
                except:
                    pass
    if internationals == 0 and non_internationals < 15:
        return
    if ("bat" in player_role or "keep" in player_role) and runs_scored_as_batsman < 100:
        return
    if "bowl" in player_role and balls_bowled_as_bowler < 100:
        return
    if "round" in player_role and (runs_scored_as_batsman < 50 or balls_bowled_as_bowler < 50):
        return
    ####################################
    Relations = soup.find('p',class_="text-uppercase gray-700 mb-0 player-card-heading text-uppercase")
    #relation_players = soup.find('p',class_="player-card-description player-description-link gray-900 m-0 player-link")
    if(Relations != None):
        if(Relations.text.lower()=='relations' or Relations.text.lower()=='relation'):
            lab.append("Relations")
            all_relations = find_relations(Relations)
            result.append([all_relations])
    ######################################
    rec = extra(u)
    lab.append("Records")
    result.append([rec])
    print(result)
    ##########################################
    records_format , records_data = format_records(u)
    for iter in range(len(records_format)):
      lab.append(records_format[iter])
      result.append([records_data[iter]])
    ###########################################
    matches = debut(debut_url)
    current_match_format = ""
    for match_detail in matches:
        match_detail = str(match_detail)
        if match_detail.startswith("Last"):
            lab.append(current_match_format + "_" + "last_appearance")
            result.append(match_detail[5:])
        elif match_detail.startswith("Debut"):
            lab.append(current_match_format + "_" + "debut")
            result.append(match_detail[6:])
        else:
            current_match_format = match_detail
    # result.append([matches])
    # lab.append("Matches")
    ##########################################
    lab.append("Teams")
    team_data = teams_data(soup)
    result.append([team_data])
    ###########################################
    analysis_and_trophy_labels, analysis_and_trophy_results = get_statistical_analysis_and_trophy_info(req)
    lab = lab + analysis_and_trophy_labels
    result = result + analysis_and_trophy_results
    ###########################################
    player_image_label, player_image_result = get_player_image(req.url)
    lab = lab + player_image_label
    result = result + player_image_result
    ###########################################
    career_span_tag = soup.find('div', class_="intl_career-desktop")
    career_span_tag_text = career_span_tag.findNext('p').text
    if len(career_span_tag_text) > 0:
        career_span_info = ""
        start_index = 0
        for i in range(len(career_span_tag_text)):
            if career_span_tag_text[i] == '1' or career_span_tag_text[i] == '2':
                start_index = i
                break
        career_span_tag_text = career_span_tag_text[start_index:]
        result.append(career_span_tag_text)
        lab.append("career_span")
    ###########################################
    search_tag_container = soup.find('div',class_="player-card__details")
    l = list()
    for job_elem in search_tag_container:
        l.append(job_elem.text)
        count+=1
    p = l[1].split("|")
    l.pop()
    l.pop()
    l = l+p
    result = result + l
    l_label = ["Player Name" , "Nationality ", "player-type"]
    lab = lab + l_label
    ##########################
      
    print('label--',len(lab))
    print('result--',len(result))
    
    lo = dict(zip(lab,result))
    lp = pd.DataFrame(lo,index=[ind])
    final_list_players.append(lp)
    print('count',count)

In [11]:
#Get individual player's "Records" information.
import re
def extra(l):
    re = requests.get(l)
    time.sleep(2)
    so = bf(re.text,"html.parser")
    res1 = so.find_all('div',class_='col-14 d-flex flex-row')
    res = so.find_all('div',class_='row bg-gray-100 class-record-row-p d-flex align-items-center')
    #print(res1)
    url_list = list()
    for i in range(len(res1)):

        pre = res1[i].find_all('h1')
        loi = res1[i].find_all('p',class_='record-class-title m-0')
        #print(loi)    
        for j in range(len(pre)):
            #print(pre[j].text)
            #print(loi[j].text)
            url_list.append(str(pre[j].text)  +str(" ") + str(loi[j].text))
    return url_list


In [12]:
def format_records(url):
  req = requests.get(url)
  soup = bf(req.text,"html.parser")
  record_sections = soup.find_all('div',class_="card content-block")
  required = ['Test Records','ODI Records','T20I Records']
  available_labels = list()
  labels_data = list()
  count = -1
  for i in range(len(record_sections)):
    name = record_sections[i].find('h5',class_="m-0")
    if(name.text in required):
      available_labels.append(name.text)
      labels_data.append([])
      count +=1
      individual_block_record = record_sections[i].find_all('a',class_="gray-900 benton-normal d-block")
      for i in range(len(individual_block_record)):
        try:
          a = individual_block_record[i].find('span',class_="m-0 benton-bold gray-1000 pr-3").text
          b = individual_block_record[i].find('span',class_="list-record-title").text
          c = individual_block_record[i].find('span',class_="m-0 ml-1 gray-600").text
          labels_data[count].append(str(a)+str(" ")+str(b)+str(" ")+str(c))
        except:
          no = 1
  return available_labels , labels_data

In [13]:
def teams_data(so):
  teams = []
  loi = so.find_all('h5',class_='m-0 ml-2 link-border-bottom player-description-link')
  for i in range(len(loi)):
    teams.append(loi[i].text)
  return teams

In [16]:
def debut(link):
  player_match = list()
  url = link
  request = requests.get(url)
  soup = bf(request.text,"html.parser")
  search_tag_container = soup.find_all('div',{'id':'debut-last-matches'})
  cp_jo = search_tag_container
  if(len(search_tag_container) != 0):
    total = cp_jo[0].find_all('div',class_="")
    for i in range(len(total)):
      p =total[i]
      a= p.find('h5',{'class':"benton-bold player-matches-subtitle d-flex justify-content-between align-items-center player_matches-dropdown m-0"})
      k = total[i].find_all('span',class_="match-event-row-debut-last white-space-nowrap")
      search_tag_container = total[i].find_all('span',{"class": "player-match-link"})
      new = total[i].find_all('div',{"class":"col-12 player-match-event-cell white-space-nowrap"})
      #print(a)
      #print(k[0])
      #print(new)
      try:
        l =a.span.decompose()
      except:
        player_match.append(a.text)
        #print(a.text)
      else:
        player_match.append(a.text)
        #print(a.text)
      for j in range(len(k)):
        try:
          temp = search_tag_container[j].text
        except:
          temp = new[j].text
        append = k[j].text + str(" ") + temp
        player_match.append(append)
        #print(k[j].text,end=" ")
        #print(search_tag_container[j].text)
  return player_match

In [17]:
import time
final_list_players = list()
for country_id in range(1,2):
  #Country based parse url
  url = "https://www.espncricinfo.com/ci/content/player/index.html?country="+str(country_id)
  req = requests.get(url)
  soup = bf(req.text,"html.parser")
  #Fetch total urls of players belonging to a country
  country_name = soup.find('span',{'id':'ciShowCountry'})
  print("-------------------------------------------------")
  print(country_name.text)
  print("-------------------------------------------------")
  search_tag_container = soup.find_all('td')
  u = list()
  for job_elem in search_tag_container:
      u.append(job_elem.find('a'))
      #result.append(job_elem.text)
  l = set(u)
  l = list(l)
  for i in range(len(l)):
      u = "https://www.espncricinfo.com"
      rl = l[i]["href"]
      link = u+rl
      print(link)
      #link = "https://www.espncricinfo.com/player/virat-kohli-253802"
      req = requests.get(link)
      time.sleep(2)
      print('player-',i,end=" ")
      ind = len(final_list_players)
      record_url = req.url+str("/tests-odi-t20-records")
      debut_url = req.url+str("/matches")
      #debut(debut_url)
      load(req,final_list_players,ind,record_url,debut_url)
      #print(record_url)
      #jk = extra(record_url)
      #print(jk)
  driver.close()

-------------------------------------------------
England
-------------------------------------------------
https://www.espncricinfo.com/ci/content/player/414990.html
player- 0 <title>Jake Ball profile and biography, stats, records, averages, photos and videos</title>
['Jacob Timothy Ball', 'March 14, 1991, Mansfield, Nottinghamshire', '30y 77d', 'Right hand bat', 'Right arm fast medium', 'Bowler', [[['JJ Ball', ' (brother),'], ['BN French', ' (uncle)']]], [['3rd Best figures in a innings on debut (5)']]]
{'The Ashes': {'Span': '2017-2017', 'Mat': '1', 'Inns': '2', 'Overs': '26.0', 'Mdns': '4', 'Runs': '115', 'Wkts': '1', 'BBI': '1/77', 'BBM': '1/115', 'Avg': '115.00', 'Econ': '4.42', 'SR': '156.00', '5w': '0', '10w': '0'}}
https://img1.hscicdn.com/image/upload/f_auto,t_gn_s_100/lsci/db/PICTURES/CMS/304300/304331.jpg
Head count 14
Head count 15
label-- 218
result-- 218
count 171
https://www.espncricinfo.com/ci/content/player/877051.html
player- 1 <title>Tom Banton profile and biography

https://img1.hscicdn.com/image/upload/f_auto,t_gn_f_345/lsci/db/PICTURES/CMS/319700/319791.png
Head count 15
Head count 14
label-- 203
result-- 203
count 170
https://www.espncricinfo.com/ci/content/player/247235.html
player- 8 <title>Chris Woakes profile and biography, stats, records, averages, photos and videos</title>
['Christopher Roger Woakes', 'March 02, 1989, Birmingham, Warwickshire', '32y 89d', 'Right hand bat', 'Right arm fast medium', 'Allrounder', '6ft 2in', [[' 1000 runs and 100 wickets ', '1st Most runs in an innings (by batting position) (95*)', '1st Best figures in a innings when on the losing side (6)', '2nd Most catches in an innings (4)', ' 1000 runs and 100 wickets ']]]
{"Basil D'Oliveira": {'Span': '2015-2020', 'Mat': '3', 'Inns': '6', 'NO': '0', 'Runs': '86', 'HS': '32', 'Avg': '14.33', 'BF': '133', 'SR': '64.66', '100s': '0', '50s': '0', '0s': '2', '4s': '13', '6s': '0'}, 'ICC World Test Champ': {'Span': '2019-2020', 'Mat': '10', 'Inns': '14', 'NO': '2', 'Runs': '

{"Basil D'Oliveira": {'Span': '2019-2020', 'Mat': '4', 'Inns': '7', 'NO': '0', 'Runs': '130', 'HS': '44', 'Avg': '18.57', 'BF': '142', 'SR': '91.54', '100s': '0', '50s': '0', '0s': '1', '4s': '22', '6s': '1'}, 'ICC World Test Champ': {'Span': '2019-2021', 'Mat': '9', 'Inns': '12', 'NO': '0', 'Runs': '192', 'HS': '44', 'Avg': '16.00', 'BF': '264', 'SR': '72.72', '100s': '0', '50s': '0', '0s': '2', '4s': '26', '6s': '4'}, 'The Ashes': {'Span': '2019-2019', 'Mat': '1', 'Inns': '2', 'NO': '0', 'Runs': '32', 'HS': '17', 'Avg': '16.00', 'BF': '39', 'SR': '82.05', '100s': '0', '50s': '0', '0s': '0', '4s': '3', '6s': '1'}, 'The Wisden Trophy': {'Span': '2019-2020', 'Mat': '3', 'Inns': '5', 'NO': '1', 'Runs': '67', 'HS': '17', 'Avg': '16.75', 'BF': '114', 'SR': '58.77', '100s': '0', '50s': '0', '0s': '0', '4s': '10', '6s': '1'}}
https://img1.hscicdn.com/image/upload/f_auto,t_gn_s_100/lsci/db/PICTURES/CMS/304300/304324.jpg
Head count 15
Head count 14
label-- 220
result-- 220
count 172
https://ww

{"Basil D'Oliveira": {'Span': '2020-2020', 'Mat': '2', 'Inns': '4', 'Overs': '113.0', 'Mdns': '40', 'Runs': '206', 'Wkts': '8', 'BBI': '5/51', 'BBM': '6/87', 'Avg': '25.75', 'Econ': '1.82', 'SR': '84.70', '5w': '1', '10w': '0'}, 'ICC World Test Champ': {'Span': '2020-2021', 'Mat': '12', 'Inns': '19', 'Overs': '385.2', 'Mdns': '81', 'Runs': '1102', 'Wkts': '33', 'BBI': '5/30', 'BBM': '8/130', 'Avg': '33.39', 'Econ': '2.85', 'SR': '70.00', '5w': '2', '10w': '0'}, 'The Wisden Trophy': {'Span': '2020-2020', 'Mat': '3', 'Inns': '4', 'Overs': '65.1', 'Mdns': '13', 'Runs': '208', 'Wkts': '5', 'BBI': '2/51', 'BBM': '3/126', 'Avg': '41.60', 'Econ': '3.19', 'SR': '78.20', '5w': '0', '10w': '0'}}
https://img1.hscicdn.com/image/upload/f_auto,t_gn_s_100/lsci/db/PICTURES/CMS/304300/304325.jpg
Head count 14
Head count 15
label-- 159
result-- 159
count 117
https://www.espncricinfo.com/ci/content/player/297433.html
player- 22 <title>Jonny Bairstow profile and biography, stats, records, averages, photos

https://img1.hscicdn.com/image/upload/f_auto,t_gn_f_345/lsci/db/PICTURES/CMS/316600/316660.png
Head count 14
Head count 15
label-- 234
result-- 234
count 172
https://www.espncricinfo.com/ci/content/player/19264.html
player- 29 <title>Liam Plunkett profile and biography, stats, records, averages, photos and videos</title>
['Liam Edward Plunkett', 'April 06, 1985, Middlesbrough, Yorkshire', '36y 54d', 'Pudsy', 'Right hand bat', 'Right arm fast', 'Bowler', '6ft 3in', 'Nunthorpe Comprehensive', [['10th Most consecutive matches missed for a team between appearances (85)', '7th Highest partnership for the ninth wicket (100)', '3rd Most consecutive matches missed for a team between appearances (74)']]]
{'The Wisden Trophy': {'Span': '2007-2007', 'Mat': '3', 'Inns': '6', 'Overs': '89.0', 'Mdns': '13', 'Runs': '315', 'Wkts': '7', 'BBI': '3/35', 'BBM': '4/60', 'Avg': '45.00', 'Econ': '3.53', 'SR': '76.20', '5w': '0', '10w': '0'}}
https://img1.hscicdn.com/image/upload/f_auto,t_gn_f_345/lsci/db/PI

{"Basil D'Oliveira": {'Span': '2019-2020', 'Mat': '4', 'Inns': '7', 'NO': '0', 'Runs': '115', 'HS': '29', 'Avg': '16.42', 'BF': '185', 'SR': '62.16', '100s': '0', '50s': '0', '0s': '0', '4s': '12', '6s': '5'}, 'ICC World Test Champ': {'Span': '2019-2021', 'Mat': '18', 'Inns': '31', 'NO': '2', 'Runs': '963', 'HS': '152', 'Avg': '33.20', 'BF': '1821', 'SR': '52.88', '100s': '1', '50s': '4', '0s': '1', '4s': '108', '6s': '14'}, 'The Ashes': {'Span': '2015-2019', 'Mat': '10', 'Inns': '18', 'NO': '0', 'Runs': '369', 'HS': '70', 'Avg': '20.50', 'BF': '775', 'SR': '47.61', '100s': '0', '50s': '1', '0s': '0', '4s': '46', '6s': '3'}, 'The Wisden Trophy': {'Span': '2015-2020', 'Mat': '9', 'Inns': '16', 'NO': '3', 'Runs': '439', 'HS': '67', 'Avg': '33.76', 'BF': '807', 'SR': '54.39', '100s': '0', '50s': '4', '0s': '2', '4s': '53', '6s': '6'}}
https://img1.hscicdn.com/image/upload/f_auto,t_gn_f_345/lsci/db/PICTURES/CMS/316600/316642.png
Head count 15
Head count 14
label-- 233
result-- 233
count 17

In [18]:
a = final_list_players[0]
df = a.T
for i in range(1,len(final_list_players)):
    lo = final_list_players[i].T
    df = pd.concat([df,lo],axis=1)
final_df = df.T
final_df.to_csv("output_players.csv")